# Solving the Boltzman equation for DM Freeze-In

## Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import mpmath 

## Usefull constants

In [2]:
m_e=0.511 #Mev
m_mu=105.658 #Mev
m_tau=1776.86 #Mev
m_charge=[m_e,m_mu,m_tau]
pi=np.pi
G = 6.71*10**-45 # MeV
Mpl = (8 * pi * G) ** (-0.5)
yinf=4.35*10**-7 # Divided by the mass of the particle
y0=0.0
geff=3

## Import $g_{*s}$

In [3]:
data = np.loadtxt('g_*s(T).txt', skiprows=1)
t_imported = data[:, 0]
g_star_s_imported = data[:, 1]
def g_star_s_interp_func(t):
    return sp.interpolate.interp1d(t_imported, g_star_s_imported, fill_value="extrapolate")(t)


## Math functions

In [4]:
def bk(n,x):
    return sp.special.kn(n,x)

def gmeijer1(x):
    x_mpf = mpmath.mpf(x)
    try:
        return float(mpmath.meijerg([[], [1]], [[-1/2, -1/2, 1/2], []], x_mpf**2))
    except (ValueError, mpmath.libmp.libhyper.HypercombError):
        return 0.0 

def gmeijer2(x):
    x_mpf = mpmath.mpf(x)
    try:
        return float(mpmath.meijerg([[], [2]], [[-1/2, 1/2, 1/2], []], x_mpf**2))
    except (ValueError, mpmath.libmp.libhyper.HypercombError):
        return 0.0 

## Number density of massive fermions

In [5]:
def int_n_e(E, m, T):
    arg = E / T
    arg = np.clip(arg, None, 700)  # Avoid overflow in exp
    return E * np.sqrt(E**2 - m**2) / (np.exp(arg) + 1)

def n_e(m,T,glib):
    int=[]
    if m>0:
        int.append(sp.integrate.quad(int_n_e,m,m*10,args=(m,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        for i in range(10):
            int.append(sp.integrate.quad(int_n_e,m*10**(i+1),m*10**(i+2),args=(m,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        int1=np.sum(int)
        return (glib/(2*np.pi**2))*int1
    elif m==0:
        int.append(sp.integrate.quad(int_n_e,0,1,args=(0,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        for i in range(10):
            int.append(sp.integrate.quad(int_n_e,10**i,10**(i+1),args=(0,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        int1=np.sum(int)
        return (glib/(2*np.pi**2))*int1
    else:
        print('Error: Negative mass')
        return 0
    

## Thermodinamical properties of "massless" fermions

In [6]:
def n_nu(T,glib):                                              # If Dirac fermions glib=4
    return 3*glib*sp.special.zeta(3)*T**3/(4*np.pi**2)         # If Majorana fermions glib=2 

def rho_nu(T,glib):                                                 # If Dirac fermions glib=4
    return (7*glib*np.pi**2*T**4)/(240)                        # If Majorana fermions glib=2 

def p_nu(T,glib):
    return (1/3)*rho_nu(T,glib)

def s_nu(T,glib):
    return (4*rho_nu(T,glib))/(3*T)

def Y_nu(T,glib):
    return n_nu(T,glib)/s_nu(T,glib)    

## Electron functions

$$ \sigma=\frac{g^2}{16\pi s^3}\sqrt{\frac{s-4m_\chi^2}{s-4m_e^2}}\left[s^2+\frac{(s-4m_e^2)(s-4m_\chi^2)}{3}+4m_\chi^2(s-2m_e^2)+4m_e^2(s-2m_\chi^2)+16m_e^2m_\chi^2    \right]$$

$$<\sigma v>=\frac{1}{8 m_\chi^4 T K_2^2(m_\chi/T)}\int_{4\text{max}(m_e^2,m_\chi^2)}^\infty ds\; \sigma(s-4m_\chi^2)\sqrt{s}K_1(\sqrt{s}/T) $$

$$ \frac{dY_e}{dx}=\frac{2<\sigma v>_en_e^2}{xHs}$$

In [7]:
def sigma_e(s,g,mchi,me):
    sigma1=g**2/(16*pi*s**3)
    sigma2=np.sqrt((s-4*mchi**2)/(s-4*me**2))
    sigma31=(s**2+((s-4*me**2)*(s-4*mchi**2))/(3))
    sigma32=4*mchi**2*(s-2*me**2)
    sigma33=4*me**2*(s-2*mchi**2)
    sigma34=16*mchi**2*me**2
    return sigma1*sigma2*(sigma31+sigma32+sigma33+sigma34)

def intsigma_e(s,g,mchi,T,me):
    intsigma1=sigma_e(s,g,mchi,me)
    intsigma2=(s-4*mchi**2)
    intsigma3=np.sqrt(s)
    intsigma4=bk(1,np.sqrt(s)/T)
    return intsigma1*intsigma2*intsigma3*intsigma4

def sigmav_e(g,mchi,T,me):
    int=[]
    n=30
    for i in range(n):
        int.append(sp.integrate.quad(intsigma_e,2**(2*i+2)*max(mchi**2,me**2),2**(2*i+4)*max(mchi**2,me**2),args=(g,mchi,T,me),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
    int.append(sp.integrate.quad(intsigma_e,2**(2*n+3)*max(mchi**2,me**2),np.inf,args=(g,mchi,T,me),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
    int2=np.sum(int)
    if int2==0.0:
        #print("Warning: sigmav_e is zero, returning 0")
        return 0.0
    c1=8*mchi**4*T*bk(2,mchi/T)**2
    return (int2)/c1


In [8]:
def dYdx_e(y,x,g,mchi,me):
    a1=sigmav_e(g,mchi,mchi/x,me)
    a2=n_e(me,mchi/x,4)
    a3=((2*pi**2)/45)*(mchi/x)**3*g_star_s_interp_func(mchi/x)
    a4=(pi*(mchi/x)**2)*np.sqrt(g_star_s_interp_func(mchi/x)/90)/Mpl
    return 2*a1*a2**2/(x*a3*a4)

In [10]:
mchi=[10]
x=np.logspace(-8,2,2000)
g_charge=np.logspace(-14,-11,100)  # Adjusted to match the range of mchi
contador=1
temp1=[]
temp2=[]
temp3=[]

for i in range(len(mchi)):
    temp2=[]
    for j in range(len(g_charge)):
        temp1=[]
        for k in range(len(m_charge)):
            y_charge=sp.integrate.odeint(dYdx_e, y0, x, args=(g_charge[j], mchi[i], m_charge[k]), rtol=1e-15, atol=1e-15, mxstep=10000)
            temp1.append(y_charge)
        temp2.append(np.sum(temp1, axis=0))
        print(f"Calculating for mchi={mchi[i]} MeV, g_C={g_charge[j]}          ", 100*contador/(len(g_charge)*len(mchi)),"%")
        contador=contador+1
    temp3.append(temp2)

Calculating for mchi=10 MeV, g_C=1e-14           1.0 %
Calculating for mchi=10 MeV, g_C=1.0722672220103253e-14           2.0 %
Calculating for mchi=10 MeV, g_C=1.1497569953977357e-14           3.0 %
Calculating for mchi=10 MeV, g_C=1.2328467394420684e-14           4.0 %
Calculating for mchi=10 MeV, g_C=1.3219411484660287e-14           5.0 %
Calculating for mchi=10 MeV, g_C=1.4174741629268077e-14           6.0 %
Calculating for mchi=10 MeV, g_C=1.5199110829529332e-14           7.0 %
Calculating for mchi=10 MeV, g_C=1.629750834620647e-14           8.0 %
Calculating for mchi=10 MeV, g_C=1.747528400007683e-14           9.0 %
Calculating for mchi=10 MeV, g_C=1.8738174228603867e-14           10.0 %
Calculating for mchi=10 MeV, g_C=2.009233002565046e-14           11.0 %
Calculating for mchi=10 MeV, g_C=2.1544346900318866e-14           12.0 %
Calculating for mchi=10 MeV, g_C=2.310129700083158e-14           13.0 %
Calculating for mchi=10 MeV, g_C=2.477076355991714e-14           14.0 %
Calculati

In [11]:
for j in range(len(mchi)):
    yvals = [temp3[j][i][-1] for i in range(len(g_charge))]
    xvals = [g_charge[i] for i in range(len(g_charge))]

    data=np.column_stack((xvals, yvals))
    np.savetxt(f"gCvsgnu_mchi_{mchi[j]:.3g}.txt", data, header="g_C Final_Yield", fmt="%.6e")

